# Preliminaries

To use the Neptune SPARQL endpoint, open the SSH tunnel using

```
ssh neptune -N
```

which reads the `~/.ssh/config` file containing:

```
host neptune-demo
 ForwardAgent yes
 User ec2-user # when using Amazon Linux
 HostName <your-ec2-address>
 IdentitiesOnly yes
 IdentityFile ~/.ssh/<your-ec2-key-file>.pem
 LocalForward 8182 <cluster-endpoint-neptune>:8182
 ```
 
 This allows access to the server through port 8182. 
 
 NOTE: at least on my computer, the Terminal window does not produce a prompt after executing this command. However, it does seem to work, since I get a response when I issue this command in a different Terminal window:
 
```
curl https://triplestore1.cluster-cml0hq81gymg.us-east-1.neptune.amazonaws.com:8182/status
```

NOTE: The dump of `AATOut_WikidataCoref.nt` from Getty was missing the period at the end of each triple. I had to add it before the file would load into Neptune.

## Configuration

In [ ]:
import requests
import json
import csv
from time import sleep
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

endpoint = 'https://triplestore1.cluster-cml0hq81gymg.us-east-1.neptune.amazonaws.com:8182/sparql'
wdqs_endpoint = 'https://query.wikidata.org/sparql'
user_agent_header = 'thesaurus_crosswalk/0.1 (https://github.com/HeardLibrary/linked-data/; mailto:steve.baskauf@vanderbilt.edu)'

sparql_request_header = {
        'Accept' : 'application/json',
#        'Content-Type': 'application/sparql-query',
        'Content-Type': 'application/x-www-form-urlencoded',
        'User-Agent': user_agent_header
    }

# Low level functions

def send_sparql_query(query_string, endpoint):
    """Sends a SPARQL query to an endpoint URL. Argument is the query string, returns a list of results."""
    # You can delete the two print statements if the queries are short. However, for large/long queries,
    # it's good to let the user know what's going on.
    #print('querying SPARQL endpoint to acquire item metadata')
    #response = requests.post(endpoint, data=query_string.encode('utf-8'), headers=sparql_request_header)
    response = requests.post(endpoint, data=dict(query=query_string), headers=sparql_request_header)
    #print(response.text) # uncomment to view the raw response, e.g. if you are getting an error
    try:
        data = response.json()
    except:
        print(response.text)

    # Extract the values from the response JSON
    results = data['results']['bindings']
    
    #print('done retrieving data')
    #print(json.dumps(results, indent=2))
    
    return results

def extract_local_name(iri):
    """Extracts the local name part of an IRI, e.g. a qNumber from a Wikidata IRI. Argument is the IRI, returns the local name string."""
    # pattern is http://www.wikidata.org/entity/Q6386232
    pieces = iri.split('/')
    last_piece = len(pieces)
    return pieces[last_piece - 1]

# Write list of dictionaries to a CSV file
def write_dicts_to_csv(table, filename, fieldnames):
    with open(filename, 'w', newline='', encoding='utf-8') as csv_file_object:
        writer = csv.DictWriter(csv_file_object, fieldnames=fieldnames)
        writer.writeheader()
        for row in table:
            writer.writerow(row)

# Read from a CSV file into a list of dictionaries
def read_dicts_from_csv(filename):
    with open(filename, 'r', newline='', encoding='utf-8') as file_object:
        dict_object = csv.DictReader(file_object)
        array = []
        for row in dict_object:
            array.append(row)
    return array


# Scripts

## Find out what graphs are in the triplestore

In [ ]:
query_string = '''select distinct ?graph where {
graph ?graph {?s ?o ?p.}
}'''

results = send_sparql_query(query_string, endpoint)
for result in results:
    graph_iri = result['graph']['value']
    print(graph_iri)

## Determine frequency of unique noun values

In [ ]:
label_parts = pd.read_csv('3d_parts.csv', na_filter=False, dtype = str)
#print(label_parts.head())

# Count the unique values of modified nouns. result is a Pandas series with the noun as the index and count as value
nouns = label_parts['noun-modified'].value_counts()

# Build a data frame from the series
#for noun, freq in nouns.iteritems():
#    id_dict = {'noun': noun, 'freq': freq}

thesaurus_ids = pd.DataFrame({'noun': nouns.index, 'freq': nouns.values})
#thesaurus_ids = thesaurus_ids.head(3).copy() # uncomment to test on a few nouns
thesaurus_ids.head()
    

## Query Neptune and the WD Query Service to match nouns to labels


Query Neptune to match each noun with Nomenclature and get any crosswalks

In [ ]:
thesauri_ids = []

for index, concept in thesaurus_ids.iterrows():
    id_dict = {'noun': concept['noun'], 'freq': concept['freq']}
    
    test_label = concept['noun'].title() # Nomenclature labels have the first letters of their labels capitalized
    print(test_label)
    query_string = '''prefix skos: <http://www.w3.org/2004/02/skos/core#>
    select distinct ?iri ?otherConcept ?prefLabel
    from <http://nomenclature_2022-02-02>
    where {
    {?iri skos:prefLabel "''' + test_label + '''"@en.}
    union
    {?iri skos:altLabel "''' + test_label + '''"@en.}
    optional {
    ?iri skos:prefLabel ?prefLabel.
    filter(lang(?prefLabel)="en")
    }
    optional {?iri skos:exactMatch ?otherConcept.}
    }
    '''

    #print(query_string)

    results = send_sparql_query(query_string, endpoint)
    found = False
    for result in results:
        iri = result['iri']['value']
        print(iri)
        nomenclature_id = extract_local_name(iri)
        id_dict['nomenclature'] = nomenclature_id
        id_dict['n_pref'] = result['prefLabel']['value']
        if 'otherConcept' in result:
            other = result['otherConcept']['value']
            if 'http://vocab.getty.edu/aat/' in other:
                print(other)
                id_dict['getty'] = extract_local_name(other)
            elif 'http://www.wikidata.org/entity/' in other:
                print(other)
                id_dict['wikidata'] = extract_local_name(other)
                
    if not('nomenclature' in id_dict):
        id_dict['nomenclature'] = ''
    if not('n_pref' in id_dict):
        id_dict['n_pref'] = ''
    if not('getty' in id_dict):
        id_dict['getty'] = ''
    if not('wikidata' in id_dict):
        id_dict['wikidata'] = ''
    print()
    thesauri_ids.append(id_dict)
    
#print(json.dumps(thesauri_ids, indent = 2))

print('done')

Query Neptune to match any nouns with Getty if they weren't already known from the Nomenclature query

In [ ]:
for concept_index in range(len(thesauri_ids)):
    # Do the AAT lookup only if it's not already known from the Nomenclature search
    # If it is found, try to also get the equivalent Wikidata concept
    if thesauri_ids[concept_index]['getty'] == '':
        noun = thesauri_ids[concept_index]['noun']
        print(noun)
        query_string = '''prefix skos: <http://www.w3.org/2004/02/skos/core#>
        prefix skosxl: <http://www.w3.org/2008/05/skos-xl#>
        select distinct ?iri ?otherConcept 
        from <http://AATOut_2Terms>
        from <http://AATOut_WikidataCoref>
        where {
        {?iri skosxl:prefLabel ?labelObject.}
        union
        {?iri skosxl:altLabel ?labelObject.}
        ?labelObject skosxl:literalForm "''' + noun +'''"@en.
        optional {?iri skos:exactMatch ?otherConcept.}
        }'''

        #print(query_string)

        results = send_sparql_query(query_string, endpoint)
        #print(json.dumps(results, indent = 2))

        found = False
        for result in results:
            iri = result['iri']['value']
            getty_id = extract_local_name(iri)
            found = True
            print(getty_id)
            thesauri_ids[concept_index]['getty'] = getty_id
            
            if 'otherConcept' in result:
                other = result['otherConcept']['value']
                if 'http://www.wikidata.org/entity/' in other:
                    print(other)
                    thesauri_ids[concept_index]['wikidata'] = extract_local_name(other)
            
        if not found:
            print('no match')
        print()

print('done')

Query Neptune for Getty preferred labels and the Wikidata Query service to find Wikidata preferred labels

In [ ]:
for concept_index in range(len(thesauri_ids)):
    print(thesauri_ids[concept_index]['noun'])
    # Do the AAT label lookup only if we know the Getty ID
    if thesauri_ids[concept_index]['getty'] != '':
        query_string = '''prefix skos: <http://www.w3.org/2004/02/skos/core#>
        prefix skosxl: <http://www.w3.org/2008/05/skos-xl#>
        select distinct ?prefLabel 
        from <http://AATOut_2Terms>
        where {

        optional {
        <http://vocab.getty.edu/aat/''' + thesauri_ids[concept_index]['getty'] +'''> skosxl:prefLabel ?labelObject.
        ?labelObject skosxl:literalForm ?prefLabel.
        filter(lang(?prefLabel)="en")
        }

        }'''

        #print(query_string)

        results = send_sparql_query(query_string, endpoint)
        #print(json.dumps(results, indent = 2))

        try:
            for result in results:
                thesauri_ids[concept_index]['g_pref'] = result['prefLabel']['value']
        except:
            thesauri_ids[concept_index]['g_pref'] = ''
    else:
        thesauri_ids[concept_index]['g_pref'] = ''
        
    # Do the Wikidata label lookup only if we know the Q ID
    if thesauri_ids[concept_index]['wikidata'] != '':
        query_string = '''prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        select distinct ?prefLabel 
        where {

        optional {
        <http://www.wikidata.org/entity/''' + thesauri_ids[concept_index]['wikidata'] +'''> rdfs:label ?prefLabel.
        filter(lang(?prefLabel)="en")
        }

        }'''

        #print(query_string)

        results = send_sparql_query(query_string, wdqs_endpoint)
        #print(json.dumps(results, indent = 2))

        try:
            for result in results:
                thesauri_ids[concept_index]['w_pref'] = result['prefLabel']['value']
        except:
            thesauri_ids[concept_index]['w_pref'] = ''
    else:
        thesauri_ids[concept_index]['w_pref'] = ''
    sleep(0.2)
        
#print(json.dumps(thesauri_ids, indent = 2))

write_dicts_to_csv(thesauri_ids, 'thesauri_ids.csv', ['noun', 'freq', 'nomenclature', 'n_pref', 'wikidata', 'w_pref', 'getty', 'g_pref'])
print('done')

# Query Neptune to find superclasses in each classification system

## Nomenclature superclasses

In [ ]:
filename = 'thesauri_ids.csv'
thesauri_ids = read_dicts_from_csv(filename)

node_list = []
edge_list = []

node_iris = []

for noun in thesauri_ids:
    if noun['nomenclature'] != '':
        print(noun['noun'])
        
        query_string = '''prefix skos: <http://www.w3.org/2004/02/skos/core#>
        select distinct ?broader_subject ?broader_object
        from <http://nomenclature_2022-02-02>
        where {
        <https://nomenclature.info/nom/''' + noun['nomenclature'] + '''> skos:broader* ?broader_subject.
        ?broader_subject skos:broader ?broader_object.
        }
        '''
        #print(query_string)

        results = send_sparql_query(query_string, endpoint)
        #print(json.dumps(results, indent=2))
        
        for result in results:
            subject_iri = result['broader_subject']['value']
            object_iri = result['broader_object']['value']
            node_iris += [subject_iri, object_iri]
            subject_id = extract_local_name(subject_iri)
            object_id = extract_local_name(object_iri)
            edge_list.append( (object_id, subject_id, {'classification_system': 'nomenclature'}) )
            #print(subject_id, object_id)

# Create non-redundant list of node IRIs
node_iris = list(set(node_iris))
#print(node_iris)

# Create a text list of node IRIs for a query
value_string = ''
for node_iri in node_iris:
    value_string += '<' + node_iri + '>\n'
#print(value_string)

query_string = '''prefix skos: <http://www.w3.org/2004/02/skos/core#>
select distinct ?subject ?prefLabel
from <http://nomenclature_2022-02-02>
where {
values ?subject {
''' + value_string + '''
}
?subject skos:prefLabel ?prefLabel.
filter(lang(?prefLabel)="en")
}
'''
#print(query_string)

results = send_sparql_query(query_string, endpoint)
#print(json.dumps(results, indent=2))

for result in results:
    subject_iri = result['subject']['value']
    label = result['prefLabel']['value']
    node_id = extract_local_name(subject_iri)
    #print(node_id, label)
    node_list.append( (node_id, {'label_string': label}) )

print('done')

In [ ]:
# directed graph
G = nx.DiGraph()
G.add_nodes_from(node_list)
G.add_edges_from(edge_list)


In [ ]:
# Create an ugly plot
fig = plt.figure(figsize=(40,10))
ax = fig.add_subplot(1, 1, 1)
node_labels = nx.get_node_attributes(G,'label_string')
nx.draw(G, with_labels=True, labels = node_labels)


In [ ]:
nx.write_gml(G, 'nomenclature.gml')